In [11]:
import pandas as pd

In [12]:
df = pd.read_csv("E:\\data\\201301\\unique_pcard\\01_13.csv")


In [13]:

# Sort data by card number and ride datetime
df = df.sort_values(by=['PCARD_NO', 'RIDE_DTIME'])
print(df)

               PCARD_NO      RIDE_DTIME  TRANS_ID  TRANSP_METHOD_CD  \
956    000E0E00039E65EB  20130113104109        59               200   
957    000E0E00039E65EB  20130113134839        60               200   
958    000E0E00039E65EB  20130113161626        61               206   
955    000E0E0003C65BGG  20130113181701        29               115   
954    000E0E0003C65BGG  20130113182559        29               200   
...                 ...             ...       ...               ...   
19516  GFEF6EBGFC50F695  20130113071823        50               105   
19517  GFEF6EBGFC50F695  20130113141201        51               105   
19514  GFEF6EBGFF30000B  20130113104202        43               115   
19513  GFEF6EBGFF30000B  20130113110313        43               200   
19515  GFEF6EBGFF30000B  20130113222439        44               200   

       TRAF_FREQ  BUS_ROUTE_ID  RIDE_TRANSP_BIZR_ID      VEHC_ID  \
956            0           NaN            211100000          NaN   
957        

In [14]:
# Convert 'RIDE_DTIME' to a proper datetime format
df['RIDE_DTIME'] = pd.to_datetime(df['RIDE_DTIME'], format='%Y%m%d%H%M%S', errors='coerce')

# Sorting the dataframe by card number and ride datetime for proper sequencing
df.sort_values(by=['PCARD_NO', 'RIDE_DTIME'], inplace=True)

# Calculate the time difference between consecutive rides
df['TIME_DIFF'] = df.groupby('PCARD_NO')['RIDE_DTIME'].diff().dt.total_seconds().div(60)  # Convert to minutes

# Identify the end of a trip using the 30-minute rule
df['END_OF_TRIP'] = df['TIME_DIFF'] > 30
df['START_OF_TRIP'] = df['END_OF_TRIP'].shift(1).fillna(True).astype(bool)  # Mark the start of new trips

# Define the origin and destination for each trip
df['TRIP_ID'] = df['START_OF_TRIP'].cumsum()  # Create a unique ID for each trip

# Display some of the rows to verify the trip identification
df[['PCARD_NO', 'RIDE_DTIME', 'TIME_DIFF', 'END_OF_TRIP', 'START_OF_TRIP', 'TRIP_ID']].head(10)


C:\Users\suman\AppData\Local\Temp\ipykernel_15232\2771964654.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['START_OF_TRIP'] = df['END_OF_TRIP'].shift(1).fillna(True).astype(bool)  # Mark the start of new trips


,PCARD_NO,RIDE_DTIME,TIME_DIFF,END_OF_TRIP,START_OF_TRIP,TRIP_ID
956,000E0E00039E65EB,2013-01-13 10:41:09,NaN,False,True,1
957,000E0E00039E65EB,2013-01-13 13:48:39,187.500000,True,False,1
958,000E0E00039E65EB,2013-01-13 16:16:26,147.783333,True,True,2
955,000E0E0003C65BGG,2013-01-13 18:17:01,NaN,False,True,3
954,000E0E0003C65BGG,2013-01-13 18:25:59,8.966667,False,False,3
1054,000E0E00309CB6EF,2013-01-13 05:41:28,NaN,False,False,3
1069,000E0E0035B5F969,2013-01-13 06:24:00,NaN,False,False,3
1074,000E0E0039E509G5,2013-01-13 05:22:29,NaN,False,False,3
1080,000E0E0039G3GC06,2013-01-13 06:13:30,NaN,False,False,3
1082,000E0E0039G3GC06,2013-01-13 06:36:35,23.083333,False,False,3


In [15]:
print(df)

               PCARD_NO          RIDE_DTIME  TRANS_ID  TRANSP_METHOD_CD  \
956    000E0E00039E65EB 2013-01-13 10:41:09        59               200   
957    000E0E00039E65EB 2013-01-13 13:48:39        60               200   
958    000E0E00039E65EB 2013-01-13 16:16:26        61               206   
955    000E0E0003C65BGG 2013-01-13 18:17:01        29               115   
954    000E0E0003C65BGG 2013-01-13 18:25:59        29               200   
...                 ...                 ...       ...               ...   
19516  GFEF6EBGFC50F695 2013-01-13 07:18:23        50               105   
19517  GFEF6EBGFC50F695 2013-01-13 14:12:01        51               105   
19514  GFEF6EBGFF30000B 2013-01-13 10:42:02        43               115   
19513  GFEF6EBGFF30000B 2013-01-13 11:03:13        43               200   
19515  GFEF6EBGFF30000B 2013-01-13 22:24:39        44               200   

       TRAF_FREQ  BUS_ROUTE_ID  RIDE_TRANSP_BIZR_ID      VEHC_ID  \
956            0           NaN 

In [20]:
trip_origin_dest = df.groupby('TRIP_ID').agg({
    'PCARD_NO': 'first',
    'RIDE_DTIME': 'first',
    'ALIGHT_DTIME': 'last',
    'RIDE_STA_ID': 'first',
    'ALIGHT_STA_ID': 'last'
}).rename(columns={
    'RIDE_DTIME': 'TRIP_START_TIME',
    'ALIGHT_DTIME': 'TRIP_END_TIME',
    'RIDE_STA_ID': 'ORIGIN_STA_ID',
    'ALIGHT_STA_ID': 'DESTINATION_STA_ID'
})
print(trip_origin_dest.head())

                 PCARD_NO     TRIP_START_TIME  TRIP_END_TIME  ORIGIN_STA_ID  \
TRIP_ID                                                                       
1        000E0E00039E65EB 2013-01-13 10:41:09   2.013011e+13           1817   
2        000E0E00039E65EB 2013-01-13 16:16:26   2.013011e+13           4209   
3        000E0E0003C65BGG 2013-01-13 18:17:01   2.013011e+13        8003104   
4        000E0E0039G3GC06 2013-01-13 22:06:32   2.013011e+13        8002147   
5        000E0E003C0595G3 2013-01-13 10:33:23   2.013011e+13           9270   

         DESTINATION_STA_ID  
TRIP_ID                      
1                    4209.0  
2                    1817.0  
3                   74935.0  
4                    1803.0  
5                 8501194.0  


In [25]:
specific_pcard='000E0E00039E65EB'

# Filter the DataFrame for rows where 'PCARD_NO' matches the specific payment card number
filtered_rows = trip_origin_dest[trip_origin_dest['PCARD_NO'] == specific_pcard]

# Display the filtered rowsa
print(filtered_rows)

                 PCARD_NO     TRIP_START_TIME  TRIP_END_TIME  ORIGIN_STA_ID  \
TRIP_ID                                                                       
1        000E0E00039E65EB 2013-01-13 10:41:09   2.013011e+13           1817   
2        000E0E00039E65EB 2013-01-13 16:16:26   2.013011e+13           4209   

         DESTINATION_STA_ID        TIME_CATEGORY  
TRIP_ID                                           
1                    4209.0        Non-Peak (NP)  
2                    1817.0  Afternoon Peak (AP)  


In [17]:
# Count the distinct TRIP_IDs for each PCard
trip_counts = df.groupby('PCARD_NO')['TRIP_ID'].nunique().nlargest(10)

# Print the top 10 PCards based on number of distinct trips
print("Top 10 PCards based on number of distinct trips:")
print(trip_counts)

Top 10 PCards based on number of distinct trips:
PCARD_NO
000E0E0G3G955FB6    12
000E0E0BGG9E9CFG    10
000E0E0GE53CFGC0    10
000E0E0G59F3E35E     9
000E0E06C55G5B35     8
000E0E0CEC9CCF0F     8
000E0E0FB0BC533C     8
000E0E0FB0BC6B65     8
000E0E0FEE6G5E06     8
000E0E0G3E95EBCE     8
Name: TRIP_ID, dtype: int64


In [18]:
def categorize_time(time):
    if time >= pd.to_datetime('06:00:00').time() and time <= pd.to_datetime('10:00:00').time():
        return 'Morning Peak (MP)'
    elif time >= pd.to_datetime('16:00:00').time() and time <= pd.to_datetime('20:00:00').time():
        return 'Afternoon Peak (AP)'
    else:
        return 'Non-Peak (NP)'

In [23]:

# Convert 'RIDE_DTIME' to datetime and extract the time part
trip_origin_dest['TRIP_START_TIME'] = pd.to_datetime(trip_origin_dest['TRIP_START_TIME'])
trip_origin_dest['TIME_CATEGORY'] = trip_origin_dest['TRIP_START_TIME'].dt.time.apply(categorize_time)
print(trip_origin_dest)

                 PCARD_NO     TRIP_START_TIME  TRIP_END_TIME  ORIGIN_STA_ID  \
TRIP_ID                                                                       
1        000E0E00039E65EB 2013-01-13 10:41:09   2.013011e+13           1817   
2        000E0E00039E65EB 2013-01-13 16:16:26   2.013011e+13           4209   
3        000E0E0003C65BGG 2013-01-13 18:17:01   2.013011e+13        8003104   
4        000E0E0039G3GC06 2013-01-13 22:06:32   2.013011e+13        8002147   
5        000E0E003C0595G3 2013-01-13 10:33:23   2.013011e+13           9270   
...                   ...                 ...            ...            ...   
10206    GFEF6EBGE090C00B 2013-01-13 19:21:32   2.013011e+13           2748   
10207    GFEF6EBGE090C00B 2013-01-13 23:13:44   2.013011e+13            325   
10208    GFEF6EBGEB9EC009 2013-01-13 22:08:41   2.013011e+13          10198   
10209    GFEF6EBGFC50F695 2013-01-13 07:18:23   2.013011e+13        9013014   
10210    GFEF6EBGFF30000B 2013-01-13 10:42:02   2.01

In [26]:
# Check initial data presence
print("Total rows in DataFrame:", len(filtered_rows))

# Check unique values in 'TIME_CATEGORY'
print("Unique time categories:", filtered_rows['TIME_CATEGORY'].unique())

# Check rows after applying Morning Peak filter
mp_rows = filtered_rows[filtered_rows['TIME_CATEGORY'] == 'Morning Peak (MP)']
print("Rows in Morning Peak:", len(mp_rows))

# Check rows after applying Afternoon Peak filter
ap_rows = filtered_rows[filtered_rows['TIME_CATEGORY'] == 'Afternoon Peak (AP)']
print("Rows in Afternoon Peak:", len(ap_rows))


Total rows in DataFrame: 2
Unique time categories: ['Non-Peak (NP)' 'Afternoon Peak (AP)']
Rows in Morning Peak: 0
Rows in Afternoon Peak: 1


In [14]:

# Identify Home and Workplace stations
# Morning Peak (MP) and Afternoon Peak (AP) time categorization
homes = filtered_rows[filtered_rows['TIME_CATEGORY'] == 'Morning Peak (MP)'].groupby('RIDE_STA_ID').size().idxmax()
workplaces = filtered_rows[filtered_rows['TIME_CATEGORY'] == 'Afternoon Peak (AP)'].groupby('ALIGHT_STA_ID').size().idxmax()

# Identify Other Activity Locations during Non-Peak (NP) hours
other_activity_locations = filtered_rows[filtered_rows['TIME_CATEGORY'] == 'Non-Peak (NP)'].groupby('ALIGHT_STA_ID').size().idxmax()

# Output the results
print(f"Home station: {homes}")
print(f"Workplace station: {workplaces}")
print(f"Other activity location: {other_activity_locations}")


ValueError: attempt to get argmax of an empty sequence